# Introduction to Value Function Iteration

In [ ]:
using Pkg
# Pkg.add(["LinearAlgebra","Gadfly","DataFrames","DataFramesMeta","Chain"]) # Run this line when running for the first time
using LinearAlgebra,Gadfly,DataFrames,DataFramesMeta,Chain


## Bruteforce Algorithm
For this demonstration I am using $\beta = 0.95, z = 1, \alpha = 0.3,$ and $\delta = 1$. This full depreciation case is particularly useful for expositional purposes becuase it admits an analytical solution for the value function. Hence, in this case, we will be able to check our iterated numerical solution against the true solution.

Given these parameter values the Bellman for our problem is,
$$V(k) = \max_{k'\in[0,k^{0.3}]} \left\{ \log\left(k^{0.3} - k'\right) + 0.95 V(k')\right\}$$

Hence, the associated Bellman operator is,
$$Tf(k) = \max_{k'\in[0,k^{0.3}]} \left\{ \log\left(k^{0.3} - k'\right) + 0.95 f(k')\right\}$$


In [ ]:
#=
Define the parameters for convenience. 
    In Julia you can type Greek letters by typing \beta + tab.
    The semicolons at the end suppress output to REPL.
=#
β = 0.95;
z = 1;
α = 0.3;
δ = 1;

#### Step 1: Create grid for the state space
For our parameter values the bounded state space is given by,
$$k\in \left[0,\left(\frac{\delta}{z}\right)^{\frac{1}{\alpha - 1}}\right] = [0,1]$$

Let's create a grid with 100 points between [0,1].

In [ ]:
X = collect(0.01:0.01:1); #state space
println(X)

#### Step 2: Guess Initial Function $V_0(k)$

Let's be really naive and set $V_0(k) = 0, \forall k$. We need to evaluate this function for all the $k\in X$

In [ ]:
V0 = [0 for k in X]; #initial guess
println(V0)

#### Step 3: Iterate Using the Bellman Operator

As mentioned above the Bellman operator is,
$$V_{n+1}(k) = \max_{k'\in[0,k^{0.3}]} \left\{ \log\left(k^{0.3} - k'\right) + \beta V_{n}(k')\right\}$$

We want to have an expression for the new function $V_{n+1}(k)$ for each $k\in X$. Therefore we will need to loop through each $k\in X$, and then for each $k$ we need to,
1. Evaluate $\log\left(k^{0.3} - k'\right) + \beta V_{n}(k')$ for each $k' \in \Gamma(k)$. *(In the algorithm below I evaluate the funciton for all $k'\in X$, but return a very negative value for infeasible k', this will ensure they are not picked as a max.)*
1. Take the max value from the previous step and assign it to $V_{n+1}(k)$. If you like, at this stage you can store the maximising k' (the $\arg \max$) as the value for the policy rule i.e. $g_{n+1}(k)$.

In [ ]:
iter = 0
iter_max = 1000
error = 9
err_tol = 1e-6

X = collect(0.01:0.01:1); # State space
V0 = fill(0.0,100); # Initial guess
V1 = [0.0 for k in X]; # updated guess (this way of generating a list is called list comprehension in Julia and Python)
#=
Coding comments:
    In Julia, sometimes it is important to use the exact type (e.g. a float number) to initiate a list, or there could be errors.
=#

mV = zeros(length(X),iter_max) # Create a matrix to save our guessed V in each iteration (for plotting later)

while error > err_tol && iter_max <= iter_max # Iterate until error is small enough or the maximal number of iterations is reached.
    
    for i = 1:100 # Loop over grid points
    
        k = X[i] # k
        objective = [X[j] <= k^α ? log(k^α - X[j]) + β*V0[j] : -Inf for j = 1:100] # Evaulate the obejective function at each k'
        V1[i] = maximum(objective) # Find the maximum of the objective function
    
    end

    error = sum((V1-V0).^2) # You could also use sup norm maximum(abs.(V1-V0))
    #=
    Coding comments:
        The dot denotes "broadcast" in Julia. It applies a function that takes one element as an argument to each element of a vector/matrix.
    =#
    
    V0 = copy(V1) # Update guess
    #=
    Coding comments:
        We need to be careful here. Julia arrays are not copied when assigned to another variable. After A = B, changing elements of B will modify A as well.
        https://docs.julialang.org/en/v1/manual/noteworthy-differences/
    =#

    iter = iter+1

    # Below are additional codes to save data points for plotting
    mV[:,iter] = V1
end
println(iter)

## Some take-aways

The code below plots each iteration. As you can see, the initial guess was pretty bad, but we still seem to have converged. If you like, play-around with the initial guess to see how the number of iterations changes.

In [ ]:
df_out = DataFrame(mV[:,1:iter],:auto)
@transform!(df_out, :k=X) # Add k values
df_out = stack(df_out,Not(:k),variable_name=:iter,value_name=:V) # Reshape to long
@transform!(df_out, :iter=parse.(Int64, SubString.(:iter,2))); # Convert iteration count to int

In [ ]:
println("Number of iterations: ", iter)
plot(df_out,x=:k,y=:V, Geom.line, color=:iter, Theme(key_position = :none), Guide.title("Value Function Iterations"))

### Numerical estimate vs. truth

As mentioned at the beginning, in the full depreciation case we in fact have an analytical expression for the value function $V(k)$. Let's see how our numerical method compares to the true function.

In [ ]:
function v_hat(k)
    a = 0.3
    b = 0.95

    return a/(1-a*b)*log(k) + 1/(1-b)*(a*b/(1-a*b)*log(a*b)+log(1-a*b))
end

V_true = [v_hat(k) for k in X]

plot(layer(@subset(df_out,:iter.==iter),x=:k,y=:V, Geom.line, color=["Numerical Iteration"],Theme(line_style=[:solid]), order=1),
    layer(x=X,y=V_true, Geom.line, color=["True Function"],Theme(line_style=[:dot]), order=2),
    Guide.title("Value Functions"))

# It might be easier to tweak figures if you RCall ggplot2 (at least I feel this way at the moment).